In [11]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# cross-validation after feature selection
# please change loading path before running
- K-fold (3)
- still look at AUC under PR due to imbalanced data

# best params save
- no woe -> ./cv_optimal_params_with_paramgrid2.p
- with woe -> ./woe_data_model/cv_optimal_params_with_paramgrid2.p

In [12]:
# import the feature 
data = pickle.load(open( "./woe_data_model/selected_feature_woe.p", "rb" ) )
X = data["X"]
y = data["y"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 10, stratify=y)

## param_grid


In [14]:
# grid data to search

param_grid1 = {
    'max_depth': [6, 9, 12],
    'learning_rate':[0.01, 0.1, 0.9],
    'gamma': [0, 0.5, 1],
    'reg_lambda': [0, 5, 10],
    'scale_pos_weight': [3, 9]
}

In [18]:
param_grid2 = {
    'max_depth': [6],
    'learning_rate':[0.1],
    'gamma': [0],
    'reg_lambda': [10, 20, 30],
    'scale_pos_weight': [3]
}

In [19]:
optimal_params = GridSearchCV(estimator = xgb.XGBClassifier(objective="binary:logistic", 
                                                            random_state = 10, 
                                                            subsample=0.8,
                                                            colsample_bytree=0.8, 
                                                            use_label_encoder=False),
                              param_grid=param_grid2,
                             scoring="average_precision",cv=3, verbose=2, n_jobs=5)
optimal_params.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[13:42:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=10,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=0.8,
                                     tree_method=None, use

In [20]:
optimal_params.best_params_


{'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 6,
 'reg_lambda': 30,
 'scale_pos_weight': 3}

In [21]:
optimal_params.best_score_

0.5564979745462204

# save the parameters

In [22]:
pickle.dump( {"optimal_params": optimal_params}, open( "./woe_data_model/cv_optimal_params_with_paramgrid2.p", "wb" ) )